<a href="https://colab.research.google.com/github/ShankarChavan/Gen-AI-LLMs/blob/main/fine-tuning/AutoTrain_LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title 🤗 AutoTrain LLM
#@markdown In order to use this colab
#@markdown - upload train.csv to a folder named `data/`
#@markdown - train.csv must contain a `text` column
#@markdown - choose a project name if you wish
#@markdown - change model if you wish, you can use most of the text-generation models from Hugging Face Hub
#@markdown - add huggingface information (token and repo_id) if you wish to push trained model to huggingface hub
#@markdown - update hyperparameters if you wish
#@markdown - click `Runtime > Run all` or run each cell individually

import os
!pip install -U autotrain-advanced > install_logs.txt
!autotrain setup --colab > setup_logs.txt

In [18]:

#@markdown ---
#@markdown #### Project Config
#@markdown Note: if you are using a restricted/private model, you need to enter your Hugging Face token in the next step.
project_name = 'my_autotrain_llm' # @param {type:"string"}
model_name = 'abhishek/llama-2-7b-hf-small-shards' # @param {type:"string"}

#@markdown ---
#@markdown #### Push to Hub?
#@markdown Use these only if you want to push your trained model to a private repo in your Hugging Face Account
#@markdown If you dont use these, the model will be saved in Google Colab and you are required to download it manually.
#@markdown Please enter your Hugging Face write token. The trained model will be saved to your Hugging Face account.
#@markdown You can find your token here: https://huggingface.co/settings/tokens
push_to_hub = False # @param ["False", "True"] {type:"raw"}
hf_token = "hf_XXX" #@param {type:"string"}
repo_id = "username/repo_name" #@param {type:"string"}

#@markdown ---
#@markdown #### Hyperparameters
learning_rate = 2e-4 # @param {type:"number"}
num_epochs = 1 #@param {type:"number"}
batch_size = 1 # @param {type:"slider", min:1, max:32, step:1}
block_size = 1024 # @param {type:"number"}
trainer = "sft" # @param ["default", "sft"] {type:"raw"}
warmup_ratio = 0.1 # @param {type:"number"}
weight_decay = 0.01 # @param {type:"number"}
gradient_accumulation = 4 # @param {type:"number"}
use_fp16 = True # @param ["False", "True"] {type:"raw"}
use_peft = True # @param ["False", "True"] {type:"raw"}
use_int4 = True # @param ["False", "True"] {type:"raw"}
lora_r = 16 #@param {type:"number"}
lora_alpha = 32 #@param {type:"number"}
lora_dropout = 0.05 #@param {type:"number"}

import os
os.environ["PROJECT_NAME"] = project_name
os.environ["MODEL_NAME"] = model_name
os.environ["PUSH_TO_HUB"] = str(push_to_hub)
os.environ["HF_TOKEN"] = hf_token
os.environ["REPO_ID"] = repo_id
os.environ["LEARNING_RATE"] = str(learning_rate)
os.environ["NUM_EPOCHS"] = str(num_epochs)
os.environ["BATCH_SIZE"] = str(batch_size)
os.environ["BLOCK_SIZE"] = str(block_size)
os.environ["WARMUP_RATIO"] = str(warmup_ratio)
os.environ["WEIGHT_DECAY"] = str(weight_decay)
os.environ["GRADIENT_ACCUMULATION"] = str(gradient_accumulation)
os.environ["USE_FP16"] = str(use_fp16)
os.environ["USE_PEFT"] = str(use_peft)
os.environ["USE_INT4"] = str(use_int4)
os.environ["LORA_R"] = str(lora_r)
os.environ["LORA_ALPHA"] = str(lora_alpha)
os.environ["LORA_DROPOUT"] = str(lora_dropout)


In [ ]:
!pip install autotrain-advanced

In [5]:
from datasets import load_dataset_builder,load_dataset
ds = load_dataset("nisaar/LLAMA2_Legal_Dataset_4.4k_Instructions")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [14]:
import pandas as pd
df = pd.concat([ds[key].to_pandas() for key in ds.keys()])
df.head()


,instruction,input,output,prompt,text
0,Analyze and explain the legal reasoning behind...,Central Inland Water Transport Corporation Ltd...,The Supreme Court in this case applied a broad...,"Below is an instruction that describes a task,...","Below is an instruction that describes a task,..."
1,Identify and summarize the key legal issues in...,Case Citation: Central Inland Water Transport ...,The key legal issues in the case Central Inlan...,"Below is an instruction that describes a task,...","Below is an instruction that describes a task,..."
2,Draft an argument appealing the decision of th...,Central Inland Water Transport Corporation Ltd...,"Honorable Supreme Court, we, on behalf of Cent...","Below is an instruction that describes a task,...","Below is an instruction that describes a task,..."
3,Identify the legal precedents used in the pres...,Case Citation: Central Inland Water Transport ...,The legal precedents used in this case are: 1....,"Below is an instruction that describes a task,...","Below is an instruction that describes a task,..."
4,Formulate a legal strategy to challenge the de...,Case: Central Inland Water Transport Corporati...,To challenge the decision of this case in a hi...,"Below is an instruction that describes a task,...","Below is an instruction that describes a task,..."


In [15]:
df.to_csv('data/train.csv',index=False)

In [19]:
!autotrain llm \
--train \
--model ${MODEL_NAME} \
--project-name ${PROJECT_NAME} \
--data-path data/ \
--text-column text \
--lr ${LEARNING_RATE} \
--batch-size ${BATCH_SIZE} \
--epochs ${NUM_EPOCHS} \
--block-size ${BLOCK_SIZE} \
--warmup-ratio ${WARMUP_RATIO} \
--lora-r ${LORA_R} \
--lora-alpha ${LORA_ALPHA} \
--lora-dropout ${LORA_DROPOUT} \
--weight-decay ${WEIGHT_DECAY} \
--gradient-accumulation ${GRADIENT_ACCUMULATION} \
$( [[ "$USE_FP16" == "True" ]] && echo "--fp16" ) \
$( [[ "$USE_PEFT" == "True" ]] && echo "--use-peft" ) \
$( [[ "$USE_INT4" == "True" ]] && echo "--use-int4" ) \
$( [[ "$PUSH_TO_HUB" == "True" ]] && echo "--push-to-hub --token ${HF_TOKEN} --repo-id ${REPO_ID}" )

> INFO    Running LLM
> INFO    Params: Namespace(version=False, train=True, deploy=False, inference=False, data_path='data/', train_split='train', valid_split=None, text_column='text', rejected_text_column='rejected', model='abhishek/llama-2-7b-hf-small-shards', learning_rate=0.0002, num_train_epochs=1, train_batch_size=1, warmup_ratio=0.1, gradient_accumulation_steps=4, optimizer='adamw_torch', scheduler='linear', weight_decay=0.01, max_grad_norm=1.0, seed=42, add_eos_token=False, block_size=1024, use_peft=True, lora_r=16, lora_alpha=32, lora_dropout=0.05, logging_steps=-1, project_name='my_autotrain_llm', evaluation_strategy='epoch', save_total_limit=1, save_strategy='epoch', auto_find_batch_size=False, fp16=True, push_to_hub=False, use_int8=False, model_max_length=1024, repo_id=None, use_int4=True, trainer='default', target_modules=None, merge_adapter=False, token=None, backend='default', username=None, use_flash_attention_2=False, log='none', disable_gradient_checkpointing=False, 